In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests 
import json
import os
from dotenv import load_dotenv
from pandas import json_normalize
import time
import seaborn as sns

In [3]:
load_dotenv()

True

In [35]:
from bs4 import BeautifulSoup as bs

In [5]:
peliculos = pd.read_csv("DATA/blockbusters.csv")
peliculos.head()

,Main_Genre,Genre_2,Genre_3,imdb_rating,length,rank_in_year,rating,studio,title,worldwide_gross,year
0,Action,Adventure,Drama,7.4,135,1,PG-13,Walt Disney Pictures,Black Panther,"$700,059,566",2018
1,Action,Adventure,Sci-Fi,8.5,156,2,PG-13,Walt Disney Pictures,Avengers: Infinity War,"$678,815,482",2018
2,Animation,Action,Adventure,7.8,118,3,PG,Pixar,Incredibles 2,"$608,581,744",2018
3,Action,Adventure,Drama,6.2,129,4,PG-13,Universal Pictures,Jurassic World: Fallen Kingdom,"$416,769,345",2018
4,Action,Comedy,NaN,7.8,119,5,R,20th Century Fox,Deadpool 2,"$318,491,426",2018


In [6]:
peliculos.tail()

,Main_Genre,Genre_2,Genre_3,imdb_rating,length,rank_in_year,rating,studio,title,worldwide_gross,year
432,Thriller,Crime,Comedy,7.1,113,6,G,Metro-Goldwyn-Mayer,The Return of the Pink Panther,"$41,833,347.00",1975
433,Music,Drama,Comedy,6.4,136,7,PG,Columbia Pictures,Funny Lady,"$39,000,000.00",1975
434,Family,Western,Comedy,6.2,100,8,G,Walt Disney Pictures,The Apple Dumpling Gang,"$36,853,000.00",1975
435,Drama,NaN,NaN,6.0,88,9,PG,Columbia Pictures,"Aloha, Bobby and Rose","$35,000,000.00",1975
436,Drama,Western,War,6.4,103,10,PG,Universal Pictures,The Other Side of the Mountain,"$34,673,100.00",1975


In [7]:
peliculos.shape

(437, 11)

In [22]:
# Filtramos aquellos valores que contienen el worlwide_gross erróneo y los que supondrán un problema al hacer concat con la otra lista que llega hasta 2020
peliculos_world_gross = peliculos[(peliculos['year'] <= 2015)]
peliculos_world_gross

,Main_Genre,Genre_2,Genre_3,imdb_rating,length,rank_in_year,rating,studio,title,worldwide_gross,year
30,Action,Adventure,Fantasy,8.0,136,1,PG-13,Walt Disney Pictures,Star Wars: The Force Awakens,"$936,662,225",2015
31,Action,Adventure,Mystery,7.0,123,2,PG-13,Universal Pictures,Jurassic World,"$652,270,625",2015
32,Action,Adventure,Sci-Fi,7.4,141,3,PG-13,Walt Disney Pictures,Avengers: Age of Ultron,"$459,005,868",2015
33,Animation,Family,NaN,8.2,94,4,PG,Pixar,Inside Out,"$356,461,711",2015
34,Action,Adventure,Mystery,7.2,140,5,PG-13,Universal Pictures,Furious 7,"$353,007,020",2015
...,...,...,...,...,...,...,...,...,...,...,...
432,Thriller,Crime,Comedy,7.1,113,6,G,Metro-Goldwyn-Mayer,The Return of the Pink Panther,"$41,833,347.00",1975
433,Music,Drama,Comedy,6.4,136,7,PG,Columbia Pictures,Funny Lady,"$39,000,000.00",1975
434,Family,Western,Comedy,6.2,100,8,G,Walt Disney Pictures,The Apple Dumpling Gang,"$36,853,000.00",1975
435,Drama,NaN,NaN,6.0,88,9,PG,Columbia Pictures,"Aloha, Bobby and Rose","$35,000,000.00",1975


In [9]:
mas_films = pd.read_csv("DATA/Github extra data/blockbusters_16fix_19_20_added.csv")
mas_films.head()

,Main_Genre,Genre_2,Genre_3,imdb_rating,length,rank_in_year,rating,studio,title,worldwide_gross,year
0,Action,Comedy,Crime,6.6,123,1,R,Columbia Pictures,Bad Boys for Life,"$426,505,244",2020
1,Action,Adventure,Comedy,6.5,99,2,PG,Paramount Pictures,Sonic the Hedgehog,"$319,715,683",2020
2,Action,Adventure,Comedy,6.1,109,3,R,Warner Bros,Birds of Prey,"$201,858,461",2020
3,Adventure,Comedy,Family,5.6,101,4,PG,Universal Pictures,Dolittle,"$245,298,647",2020
4,Drama,Horror,Mystery,7.1,124,5,R,Universal Pictures,The Invisible Man,"$143,151,000",2020


In [25]:
mas_films_filt = mas_films[(mas_films['year'] >= 2016)]
mas_films_filt.tail()
# Nos aseguramos que el df no coje valores del 2015

,Main_Genre,Genre_2,Genre_3,imdb_rating,length,rank_in_year,rating,studio,title,worldwide_gross,year
45,Action,Comedy,Adventure,8.0,103,9,R,20th Century Fox,Deadpool,"$782,612,155",2016
46,Animation,Action,Adventure,8.0,108,4,PG,Walt Disney Pictures,Zootopia,"$1,023,784,195",2016
47,Action,Adventure,Sci-Fi,6.5,151,7,PG-13,Warner Bros,Batman v Superman: Dawn of Justice,"$873,634,919",2016
48,Action,Adventure,NaN,6.1,123,10,PG-13,Warner Bros,Suicide Squad,"$746,846,894",2016
49,Adventure,Family,Fantasy,7.3,132,8,PG-13,Warner Bros,Fantastic Beasts and Where to Find Them,"$814,037,575",2016


In [12]:
# pd.concat para añadir csv de github encontrado en keppler dónde un buen samaritano ha añadido datos corregigos de groos made in films 
merged_peliculos =  merged = pd.concat([mas_films_filt, peliculos_world_gross])

In [13]:
merged_peliculos.shape

(457, 11)

In [31]:
# Empezamos a hacer el API con la página de boxofficemojo para ver también el domestic box office y ver el comportamiento o gustos norteamericanos
box_url = "https://www.boxofficemojo.com/year/2020/?ref_=bo_yl_table_2"

In [32]:
res_box = requests.get(box_url)
res_box

<Response [200]>

In [33]:
print(res_box.headers)

{'Server': 'Server', 'Date': 'Mon, 08 Nov 2021 23:14:30 GMT', 'Content-Type': 'text/html;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'x-amz-rid': '3FHJCFS60T5MXC1HAY2C', 'Cache-Control': 'no-cache, no-store, max-age=0', 'Content-Language': 'en-US', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding,Content-Type,Accept-Encoding,X-Amzn-CDN-Cache,X-Amzn-AX-Treatment,User-Agent', 'Permissions-Policy': 'interest-cohort=()'}


In [34]:
"2019" in str(res_box.content)
# Miramos a ver si otros años aparte de 2020 que es el que aparece en el url son accesibles

True

In [37]:
soup = bs(res_box.content, 'html.parser')
# Lammamos soup a ala variable que va a parse el contenido de la html

In [41]:
titulos_peliculos = soup.find_all("td", class_= "a-text-left mojo-field-type-release mojo-cell-wide")

In [43]:
titulos_peliculos[0]
# Encontramos el primer elemento de la lista, la película Bad Boys for Life

<td class="a-text-left mojo-field-type-release mojo-cell-wide"><a class="a-link-normal" href="/release/rl1182631425/?ref_=bo_yld_table_1">Bad Boys for Life</a></td>

In [44]:
titulos_peliculos[0].select('a')
# Selecciona solo la parte 'a class' pero queremos más

[<a class="a-link-normal" href="/release/rl1182631425/?ref_=bo_yld_table_1">Bad Boys for Life</a>]

In [45]:
titulos_peliculos[0].select('a')[0]
# Python cree por los corchetes que esto es una lista sin comas, por lo que asumimos qu el único elemnto en nuestra lista es el de index = 0

<a class="a-link-normal" href="/release/rl1182631425/?ref_=bo_yld_table_1">Bad Boys for Life</a>

In [46]:
titulos_peliculos[0].select('a')[0].string
# El uso de string method hace que solo cojamos los strings en el html

'Bad Boys for Life'

In [54]:
#Necesitamos una función que nos llame a todos los títulos que queremos, específicamente los top 10 de cada año""" 

pescamos_peliculos = []
for peli in titulos_peliculos:
    pescamos_peliculos.append(peli.select('a')[0].string)

In [52]:
pescamos_peliculos

['Bad Boys for Life',
 '1917',
 'Sonic the Hedgehog',
 'Jumanji: The Next Level',
 'Star Wars: Episode IX - The Rise of Skywalker',
 'Birds of Prey',
 'Dolittle',
 'Little Women',
 'The Invisible Man',
 'The Call of the Wild',
 'Onward',
 'Tenet',
 'Knives Out',
 'Frozen II',
 'Spies in Disguise',
 'The Gentlemen',
 'Just Mercy',
 'The Croods: A New Age',
 'Parasite',
 'Fantasy Island',
 'Uncut Gems',
 'The New Mutants',
 'Wonder Woman 1984',
 'Like a Boss',
 'The Grudge',
 'Unhinged',
 'The Photograph',
 'The War with Grandpa',
 'Underwater',
 'The Turning',
 'Gretel & Hansel',
 'Honest Thief',
 'My Hero Academia: Heroes Rising',
 'Bombshell',
 'The Way Back',
 'Brahms: The Boy II',
 'Jojo Rabbit',
 'Impractical Jokers: The Movie',
 'Ford v Ferrari',
 'Emma.',
 'Bloodshot',
 'I Still Believe',
 'Come Play',
 'Let Him Go',
 'Freaky',
 'Downhill',
 'Weathering with You',
 'Cats',
 'The Hunt',
 'The Rhythm Section',
 'Monster Hunter',
 'A Beautiful Day in the Neighborhood',
 'Hocus Pocus

In [17]:
#Si lo haces de la manera que pone en Medium pues tendrás que sacar todos los nombres de las pelis con sus respectivos años y además borrar esos años que no están en la página 1975/1976


In [18]:
type(peliculos)

pandas.core.frame.DataFrame

In [19]:
# esto se tiene que modificar
nan_pelis = peliculos.isnull().sum()
nan_pelis

Main_Genre           0
Genre_2             29
Genre_3            141
imdb_rating          0
length               0
rank_in_year         0
rating               0
studio               0
title                0
worldwide_gross      0
year                 0
dtype: int64